# 1. Transaction Kavramı ve OLTP Sistemler

* SQL Server üzerinde yaptığımız her bir işlem (insert, update ve delete) bir **transaction**dır. 
    * Tek başlarına da bir transactiondır. Birden fazla sıralı şekilde devam eden işlemler de bir transactiondır.
* **OLTP** (Online transaction processing), transactional yapıyı destekleyen sistemlerdir.
    * SQL Server da bir OLTP sistemdir.
    
### EFT ile Para Transferi İşlemi

<img src = "https://i.hizliresim.com/9lt4nyh.png">

* Temel anlamda sıkıntısız şekilde yürüdüğünü varsaydığımız bu sistem bu şekilde çalışır.
* Herhangi bir yerinde sıkıntı olsa;
    * Mesela ilk işlem gerçekleşip çıkış hareketi girilse ve bakiye de azaltılsa ancak üçüncü adımda insert işlemi gerçekleşemese ve bu diğer adımı da etkilese şöyle bir durum olur;
    
<img src = "https://i.hizliresim.com/fqbp9cl.png">

* Bu noktada iki taraf da haklıdır çünkü sistemde teknik bir sıkıntıdan dolayı böyle bir durum oluşmuştur.
* Olması gereken şudur;
    * Problem her zaman oluşabileceği için burada yapılması gereken her zaman bir problem oluşmasına karşın **sistemi geriye alabilmek**tir. İşte OLTP yapısı budur.
    * Bu OLTP yapısındaki her bir işlem transactiondır ve bir yerde bir sıkıntı olduğunda sistem kendisini geriye alır. 
    * Modern veritabanlarının hepsi bu sistemi destekler.
* Bu işlem veri tabanı yazılımcısının insiyatifine kalmış bir işlem değildir. Veri tabanı bazında ayarlanmalıdır.

### SQL Server OLTP Yapısı

* SQL Server'da bir user databasei oluşturulduğu zaman bir **MDF** ve bir de **LDF** dosyası oluşur.

<img src = "https://i.hizliresim.com/hrwiey0.png">

* Bu tarz sorunların önüne geçebilmek adına SQL Server'da bu tür yapılar bu iki dosya üzerinde yürür.

* Örneğin doğru bir **INSERT** işlemi gerçekleştiriyoruz.
    * Insert ile eklenen verilerin tutulduğu 8 kblik pageler önce *LDF* dosyasına yazılır. 
    * Ardından *LDF*'den *MDF* dosyası üzerine yazılır ve herhangi bir sıkıntı yoksa sistem **COMMIT** edilir ve veri *MDF* içerisinde kalır.
    
* Örneğin yanlış bir **INSERT** işlemi gerçekleştiriyoruz.    
    * Insert ile eklenen verilerin tutulduğu 8 kblik pageler önce *LDF* dosyasına yazılır. 
    * Ardından *LDF*'den *MDF* dosyası üzerine yazılır ve eğer bir sıkıntı varsa sistem **ROLLBACK** edilir ve veri *MDF* içerisinde kalmadan tekrar *LDF*e aktarılır, devamında da temizlenir. Böylelikle hiç olmamış gibi görülür.
    
* EFT ile Para Transferi İşlemi örneğinde adımlar sırasıyla *insert*, *update*, *insert* ve *update* işlemleridir.
    * Bu art arda gelen 4 işlem OLTP mimarisi ile çalışır.
    * Bunlardan herhangi birinde bir hata ile karşılaşıldığında sistem hepsini birden geri alır.

# 2. DB Oluşturma

OLTP mimarisini incelemek üzere bir database (**BANK**) oluşturalım.
* Database oluşurken database ile aynı isimde bir MDF, ve log eklentili bir LDF dosyası oluşturulur.

Amacımız; **AHMET isimli müşterinin ÖMER isimli müşteriye 1.000 TL para göndermesi.**

### Tablolar

#### CUSTOMERS

<img src = "https://i.hizliresim.com/kvcy2vx.png">

* Müşterilerin id ve isim bilgilerinin tutulduğu tablodur.

#### ACCOUNTS

<img src = "https://i.hizliresim.com/5hj963p.png">

* Müşterilerin hesaplarının numaralarının, hesap isimlerinin ve hesaplarının döviz türlerinin tutulduğu tablodur.

#### ACCOUNTBALANCE

<img src = "https://i.hizliresim.com/li0kzpp.png">

* Müşterilerin hesaplarındaki bakiyeyi ve bakiyenin döviz türünü tutan tablodur.

#### MONEYTRANSACTIONS

<img src = "https://i.hizliresim.com/qnwppo3.png">

* Müşterilerin para hareketlerini tutan tablodur.
    * *TRANTYPE*, 1 ise giriş, 2 ise çıkış hareketidir.
    * *EFTCODE1* ve *EFTCODE2* paranın A bankasından B bankasına hareket ederken takip numaralarının tutulduğu alandır.
    
### Veritabanına bakış

<img src = "https://i.hizliresim.com/2le0rfs.png">

* AHMET'in HESAPNO1 nolu hesabının adı AHMETIN HESABI ve içerisinde 1000 TL para var.
* OMER'in HESAPNO2 nolu hesabının adı OMERIN HESABI ve içerisinde 0 TL para var.

# 3. Banka Transaction Uygulaması

<img src = "https://i.hizliresim.com/b9vy6p8.png">

* Planlanan senaryoda işlemlerde herhangi bir sıkıntı çıkmadığında sonuç bu şekilde oluşur.

### Hata
* Diyelim ilk iki adım gerçekleşti ancak üçüncü adımda *EFTCODE2* karakter sınırını geçtiği için hata verdi.
    * Hatanın bu olmasına gerek yok insert işlemini engelleyen herhangi bir hata da olabilir.
* Bu noktada diğer adıma da geçilemez ve tablolar şu şekilde kalır.

<img src = "https://i.hizliresim.com/4kx3t6m.png">

* Ahmet'e göre Ömer yalancı, Ahmet parayı gönderdim diyor, Ömer para yok diyor. Ömer'e göre Ahmet yalancı, para bende değil ben almadım göndermedi o diyor.

* Bu tarz sorunların önüne geçmek adına bu işlemler teker teker görülmek yerine bir **transaction** bloğu içerisinde görülürse, bir hata alındığında geriye aldırılabilir.

## <CODE>BEGIN TRAN</CODE>

<img src = "https://i.hizliresim.com/jlm0job.png">

* Transaction bu şekilde başlatılabilir ve veriler adımlara uygun şekilde eklenip, güncellenerek değiştirilebilir.
* Şu anda transaction başlatıldı ve bu bilgiler *LDF* dosyasının içerisinde bulunuyor.
* Eğer blok sonuna <code>ROLLBACK</code> veya <code>ROLLBACK TRAN</code> denilirse tablolar transaction öncesi haline geri döner.

### Hata varsa;

* Hata tespiti için her bir işleme *hata varsa* koşulu eklenir.
* SQL Server'da global değişkenler **<code>@@</code>** ile tanımlanır.

<img src = "https://i.hizliresim.com/k3zxdr2.png">

* Örneğin **<code>@@ERROR</code>** değişkeni hata varsa 0'dan büyük bir değer döndürür ve buna göre her bir transaction için rollback işlemi gerçekleştirilebilir.

<img src = "https://i.hizliresim.com/8kydjj5.png">

* Eğer hiçbir hata yoksa da zaten transactionı commit edebilirsin.

<img src = "https://i.hizliresim.com/ge3rdmu.png">

* Herhangi bir işlem hata verdiği takdirde önceki işlemler başarıyla geçilse bile ilgili işlem hata verdiğinde rollback ile sistem transaction öncesine alınır.

<img src = "https://i.hizliresim.com/o5263cv.png">

**Transaction mekanizması tam olarak bu işe yarar. Toplu işlemlerin otomatik olarak tamamlanmasını ya da geri alınmasını sağlar.**

**TRANSACTION, <CODE>BEGIN TRAN</CODE> ile başlar, <code>COMMIT TRAN</code> ile MDF'e kaydedilir, <code>ROLLBACK TRAN</code> ile LDF dosyasından çıkılıp sistem eski haline geri döndürülür.**

# 4. Transaction ile Tabloyu Kilitleme

* SQL Server'da çalıştırılan her türlü tek satır işlem de aslında transactiondır.
* Bu bağlamda tek satırlık işlemlerde BEGIN TRAN, ROLLBACK TRAN ve COMMIT TRAN gibi komutlara gerek yoktur. Çünkü bütün işlemler otomatik gerçekleşir.
* Ancak toplu işlemlerde transaction kontrolü bizde olur.

<img src = "https://i.hizliresim.com/mvow0hf.png">

* *ITEMS* tablosundaki *PILLI SESLI UCAK* ürününün birim fiyatını değiştirmeyi deneyelim.
* Bu şekilde yazıldığına TRANSACTION başlar ancak herhangi bir ROLLBACK veya COMMIT edilmediği için TRANSACTION kapanmaz ve tabloya başka bir yerden erişim sağlanamaz.
* Bu noktada eğer tablomuzda bir TRIGGER varsa o da etkilenir ve işlemlerine devam eder. Bir log tablosuna kayıt atabilir.
* Ancak TRANSACTION, ROLLBACK edildiği takdirde hem ürünün tablodaki değeri eski haline gelir hem de TRIGGER hiç çalışmamış gibi olur.
    * **Triggerlar, transactionların bir parçasıdır.**
    * Aynı şekilde triggerda bir hata çıksa transaction gerçekleşemeyecek ve otomatik olarak rollback yapılacaktı.
    * Transaction rollback olduğunda, triggerın çalıştırdığı işlem de otomatik olarak rollback olur.
* Tek bir kayıt üzerinden yaptığın bu işlem tüm tabloyu etkiler ve tablo başka bir kullanıcı tarafından <code>SELECT * FROM ITEMS</code> diyerek bile çekilemez, tablo kilitlenmiş haldedir ve buna **LOCK** denir.

<img src = "https://i.hizliresim.com/htt06ba.png">

* Eğer kilide takılmadan tabloyu görmek istiyorsan <code>WITH (NOLOCK)</code> komutunu sorguna eklersin.
* Bu noktada tablonun commit veya rollback edilsin ya da edilmesin transactionlı halini görürsün.
    * SQL Server'a göre bir transaction başlamışsa onun commit edilme ihtimali rollback edilme ihtimalinden daha yüksektir. O yüzden transaction commit edilmiş gibi davranır.
    * SQL Server bu sorguyu MDF dosyadan değil LDF dosyadan okur.
* Ancak kendi sessionında bu özelliği kullanmadan doğrudan LDF'den görebilirsin.
    
<img src = "https://i.hizliresim.com/s0ki4iv.png">

* <code>DBCC OPENTRAN</code> komutu, halihazırda çalışan transaction var mı? varsa hangi kullanıcı çalıştırıyor? diye bakar.

